- `tf.Transform`
- Define a preprocessing function, a logical description of the pipeline that transforms the raw data into the data used to train a machine learning model.
- Show the Apache Beam implementation used to transform data by converting the preprocessing function into a Beam pipeline.

# 1.定义一个 preprocessing function

In [2]:
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

In [ ]:
def preprocessing_fn(inputs):
    x = inputs["x"]
    y = inputs["y"]
    s = inputs["s"]
    x_centered = x - tft.mean(x)
    y_normalized = tft.scale_to_0_1(y)
    s_integerized = tft.compute_and_apply_vocabulary(s)
    x_centered_times_y_normalized = x_centered * y_normalized
    
    return {
        "x_centered": x_centered,
        "y_normalized": y_normalized,
        "x_centered_times_y_normalized": x_centered_times_y_normalized,
        "s_integerized": s_integerized,
    }

# 2.Apache Beam 应用

In [4]:
raw_data = [
    {"x": 1, "y": 1, "s": "hello"},
    {"x": 2, "y": 2, "s": "world"},
    {"x": 3, "y": 3, "s": "hello"},
]
raw_data_metadata = ...
transformed_dataset, transform_fn = (
    (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn)
)
transformed_data, transformed_metadata = transformed_dataset

transformed_data, transform_fn = (my_data | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn))
transform_fn = my_data | tft_beam.AnalyzeDataset(preprocessing_fn)
transformed_data = (my_data, transform_fn) | tft_beam.TransformDataset()

In [ ]:
real_valued_columns = [feature_column.real_valued_column(key) for key in NUMERIC_FEATURE_KEYS]
one_hot_columns = [
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_file(
            key = key,
            vocabulary_file = tf_transform_output.vocabulary_file_by_name(vocab_filename = key)
        )
    ) for key in CATEGORICAL_FEATURE_KEYS
]
estimator = tf.estimator.LinearClassifier(real_valued_columns + one_hot_columns)